In [1]:
%cd ..

/Users/mitoura/Desktop/Repositories/rag--projects


In [2]:
from src.services import AzureOpenaiService
from src.utils import Settings

sets = Settings()
azure_openai_service = AzureOpenaiService(sets)
llm = azure_openai_service.get_llm()
embeddings = azure_openai_service.get_embeddings()

In [3]:
# install faiss cpu
%pip install -qU faiss-cpu


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np

corpus = [
    "O gato está dormindo no sofá.",
    "O cachorro está latindo no quintal.",
    "A Lua gira ao redor da Terra.",
    "Estou aprendendo Python para ciência de dados.",
    "O céu está limpo hoje.",
    "Machine learning é uma área fascinante.",
    "Hoje é um bom dia para correr no parque."
] # examples texts in portuguese

In [5]:
corpus_embeddings = np.array(embeddings.embed_documents(corpus)).astype("float32")

In [6]:
# example creating a FAISS index

import faiss

dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings) # add corpus embeddings to the index

In [7]:
# search for similar texts

query = "Estou aprendendo inteligência artificial"
query_embedding = embeddings.embed_query(query)
query_embedding = np.array([query_embedding]).astype("float32")
k = 3  # top-k
distances, indices = index.search(query_embedding, k)

In [8]:
for i in range(k):
    print(f"{i+1}. {corpus[indices[0][i]]} (distância: {distances[0][i]:.4f})")

1. Estou aprendendo Python para ciência de dados. (distância: 0.1834)
2. Machine learning é uma área fascinante. (distância: 0.2288)
3. O gato está dormindo no sofá. (distância: 0.4105)


In [9]:
# example to generate a local vdb with faiss

from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS


texts = [
    "O gato está dormindo no sofá.",
    "O cachorro está latindo no quintal.",
    "A Lua gira ao redor da Terra.",
    "Estou aprendendo Python para ciência de dados.",
    "O céu está limpo hoje.",
    "Machine learning é uma área fascinante.",
    "Hoje é um bom dia para correr no parque."
] # examples texts in portuguese

docs = [Document(page_content=t) for t in texts]

# crate a FAISS index in memory
vdb = FAISS.from_documents(docs, embeddings)
vdb.save_local("faiss_index")

In [10]:
# loag vdb from local storage
vdb = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [11]:
# search
query = "Estou aprendendo inteligência artificial"
resultados = vdb.similarity_search(query, k=3)

for i, r in enumerate(resultados):
    print(f"{i+1}. {r.page_content}")

1. Estou aprendendo Python para ciência de dados.
2. Machine learning é uma área fascinante.
3. O gato está dormindo no sofá.
